In [1]:
import youtube_dl
import os
from pprint import pprint
from pathlib import Path
import pandas as pd
import subprocess
import sys
import openpyxl
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import shutil
import re
from tqdm import tqdm
from moviepy.editor import *
from converter import Converter
import glob
import cv2

In [2]:
df = pd.read_excel ('C:\\Users\\DSPLAB\\Desktop\\Alroy Data\\Youtube-dl proj\\balanced_train_segments_propertext.xlsx',header=3) #takes the 3rd row as the header column
#header number is the row number to be considered as header
selection=df['# YTID']
print (selection)
ytlink= "youtube-dl https://www.youtube.com/watch?v="
print(ytlink)

0        --PJHxphWEs
1        --ZhevVpy1s
2        --aE2O5G5WE
3        --aO5cdqSAg
4        --aaILOrkII
            ...     
22155    zyqg4pYEioQ
22156    zz0ddNfz0h0
22157    zz8TGV83nkE
22158    zzlK8KDqlr0
22159    zzya4dDVRLk
Name: # YTID, Length: 22160, dtype: object
youtube-dl https://www.youtube.com/watch?v=


In [ ]:
#one time use
#to check which youtube video was last downlaoded and get its index to key into selection[:]
#manual: step1 see CMD.txt file the last video downloaded, step2: seacrh this ID in balance excel sheet, take its row number minus 4, step3: input this number into selection[:]
#the very first video hasnt been downloaded yet
selection[3172:]

In [22]:
#TO DOWNLOAD!! clips from YTID 
os.chdir(r'D:\Alroy youtube proj')
for i in selection[3172:]: #fill in [:]. Continue off from the last video you downloaded, the index of the next video to download onwards
    biglink="youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v="+i #decided to download all future files in .mp4 format to prevent slicing errors
    print(biglink, 'is downloading rn') #prints the file that is in the process of being downloaded
    os.system(biglink+" >> YTcommand_prompt.txt") #it adds on text to existing .txt file

youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6QAZJ4H_5rA is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6QCcY4zycIg is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6QHfXuVLhe0 is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6QcAmUpe8FQ is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6QnEwCnKdP4 is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6Qzlke1phUs is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6R7-SJPID_I is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v=6R7__52vkro is downloading rn
youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m

KeyboardInterrupt: 

In [ ]:
#One time use, manually individual re-downlaoding of corrupted videos
os.chdir(r'D:\Alroy youtube proj')
biglink="youtube-dl -f bestvideo[ext=mp4]+bestaudio[ext=m4a]/best https://www.youtube.com/watch?v="+"___replace this with 11 charcater YTID____" #decided to download all future files in .mp4 format to prevent slicing errors
print(biglink, 'is downloading rn') #prints the file that is in the process of being downloaded
os.system(biglink) #it adds on text to existing .txt file

In [4]:
#OLD finding out which YT video ID has been downloaded from the CMD text file
my_file = open(r"D:\Alroy youtube proj\YTcommand_prompt.txt", "r")
content_list = my_file.readlines()

#finding the word 'Destination' as keyword to show which YTID has really been downloaded
YTdownloaded = [k for k in content_list if 'Destination' in k]

#a list of YTIDs that are truly/sucessfully downloaded based off CMD text file
pear=[]
for g in YTdownloaded:
#     if g == "Gears of war 3......por éso odio a Big Rig Dizzy.....¬¬-5hZDNqoi15I.mp4": #the only file tht has no f140 in its (__filename__.f140.mp4)
#         continue
    apple=g.rsplit('.', 2) #this line cant detect Gears of war 3 being downloaded
    #pprint(apple) #splitting each string into 3 parts, one by one in the for-loop
    slicedapple=apple[0][-11:]
    pear.append(slicedapple)

#to remove duplicates
pear=list(dict.fromkeys(pear))
print(pear) #final list of YTID that sucessfully downloaded

print(len(pear)) #pear is a list consisting of the number of videos downloaded successfully, rmbr check folder the same no. of items

In [3]:
#worms, a new list of YTIDs of the files that are successfully downloaded, not linked to CMD file
new_list=os.listdir(r'D:\Alroy youtube proj')
new_list.remove('Snippets')
new_list.remove('YTcommand_prompt.txt')
print(len(new_list))
worms=[]
for flies in new_list:
    maggot=flies.rsplit('.', 2)
    
    if maggot[-2].startswith('f') and len(maggot[-2])==4:
        pupa=maggot[-3][-11:]
    else:
        pupa=maggot[-2][-11:]
        
    worms.append(pupa)
print(len(worms))

5505
5505


In [91]:
#one time use
#to check if gears of war is in List of downloaded videos (pear), IT IS downlaoded but python didnt extract its YTID because of its string format, nvm later i manually clip
file='5hZDNqoi15I'
positionInList=0
for i in pear:
    idx=i.find(file)
    positionInList+=1
    if idx!=-1:
        name=i[idx:idx+11]
        print(name)
        print(positionInList)

In [4]:
#reading the main balanced_segments excel file as dataframe
startend = pd.read_excel (r'C:\Users\DSPLAB\Desktop\Alroy Data\Youtube-dl proj\balanced_train_segments_propertext.xlsx',header=3) #takes the 3rd row as the header column
startend[['# YTID', ' start_seconds', ' end_seconds']]

,# YTID,start_seconds,end_seconds
0,--PJHxphWEs,30.0,40.0
1,--ZhevVpy1s,50.0,60.0
2,--aE2O5G5WE,0.0,10.0
3,--aO5cdqSAg,30.0,40.0
4,--aaILOrkII,200.0,210.0
...,...,...,...
22155,zyqg4pYEioQ,20.0,30.0
22156,zz0ddNfz0h0,30.0,40.0
22157,zz8TGV83nkE,80.0,90.0
22158,zzlK8KDqlr0,370.0,380.0


In [5]:
#df_reduced is the filtered 'startend' dataframe as it consist only of the videos that are sucessfully downloaded, to get start and end timings
df_reduced = startend[startend['# YTID'].str.contains('|'.join(worms))] #something wrong with this matching code
print(df_reduced[['# YTID', ' start_seconds', ' end_seconds']])
print(len(df_reduced)) #to see the number of rows in the reduced dataframe matches the length of the list in worms

           # YTID   start_seconds   end_seconds
0     --PJHxphWEs            30.0          40.0
2     --aE2O5G5WE             0.0          10.0
5     --cB2ZVjpnA            30.0          40.0
6     --ekDLDTUXA            30.0          40.0
7     -0DLPzsiXXE            30.0          40.0
...           ...             ...           ...
6426  ExA9lhy0_cc            30.0          40.0
6427  ExAC7QYH9lg            30.0          40.0
6428  ExKxdQ_Kph0           200.0         210.0
6429  ExndFnmXo0g             0.0          10.0
6430  ExqedxdXuBc            70.0          80.0

[5505 rows x 3 columns]
5505


In [7]:
#Returns a set of files between Snippets and Main download folder that has NOT been sliced, it worked!
#Using find function
longname=os.listdir(r"D:\Alroy youtube proj") #List of long names
longname.remove("Snippets")
longname.remove("YTcommand_prompt.txt")
hj=os.listdir(r"D:\Alroy youtube proj\Snippets")
shortname=[] #Compare list of short names
for i in hj:
    yu=i.rsplit('.',1)
    shortname.append(yu[-2])
print("no. of shortnames:", len(shortname))
#print(shortname) #no duplicates, i checked
    
actuallySliced=[]
for g_g in longname: #longname
    for h_h in shortname:  #shortname, keyword
        idx=g_g.find(h_h) #returns the index of where the keyword is found in the string
        if idx!= -1: #only can put != -1, if not each iteration it tried to macth will append to the list
            actuallySliced.append(g_g)
print("no. of actuallySliced:", len(actuallySliced)) #number of actually sliced should be = no. of shortnames
#print(actuallySliced)

#Comparing both lists: actuallySliced and longname. Finding the contrasting differences = unsliced videos, both lists must be IDENTICAL to one another, onyl finds the exact difference
longset=set(longname)
shortset=set(actuallySliced)
unslicedset=shortset.symmetric_difference(longset)
# print(unslicedset)
print("no. of unsliced file:", len(unslicedset)) #always check: take this number + number of snippets i have = total number of files in downloaded folder
#idk why but actuallySlicedfile should be = shortnames. I havent detective yet

no. of shortnames: 2777
no. of actuallySliced: 2754
no. of unsliced file: 2751


for string in longname: #some shortnames cant be found within the long names, returns idx== -1, idk why, impossible because the snippets come FROM the long names.
    for smol in shortname:
        idx=string.find(smol)
        if idx== -1:
            continue
    print(string)

#to remove duplicates
cheggerinos=list(dict.fromkeys(shortname))
print(cheggerinos) #final list of YTID that sucessfully downloaded

print(len(cheggerinos)) #pear is a list consisting of the number of videos downloaded successfully, rmbr check folder the same no. of items

In [74]:
#one time use
#finding the last index position of the file where the slicing stops
def ifSliceError(): #returns the Index position of the errored file in the list you were processing, returns the filename
    counter=0
    for i in lastslice():
        u=i.find('Be8u83UbvDo') #replace this arugument with the last video successfully sliced
        if u==-1:
            counter+=1
        elif u!=-1:
            return counter, i
        
ifSliceError()

(1907, 'Gears of war 3 retro lancer sound test.-Be8u83UbvDo.mp4')

In [ ]:
lastslice()[1905:]

z="blahblahwow736485hgf71.yoyo.mp4" #take -3
z=z.split('.')
z[-3][-11:]

#case3: normal case, most common
cstring="(ACE) Atlantic Coast Express clips.- Saltash and Plymouth-080914-1yRMYH8cSFM.yoyo.mp4" #take -2
print(len(cstring.rsplit('.',3)))
c=cstring.rsplit('.',2)
c[0][-11:]

def keyword(YTID, data):   
    for h in YTID:
        for i in data:
            idx=i.find(h)
            if idx> -1: #idx is index
                found=i[idx:idx+11]
                return found
            else: continue

#testing .find() function
namelist = ["(ACE) Atlantic Coast Express clips.- Saltash and Plymouth-080914-1yRMYH8cSFM.yoyo.mp4",
            "This-video-is-titled-1yRMYH8cSFM-cos-i-want-to-troll-you-actually-its-idx-is-asdfghjklgh.yoyo.mp4",
            "This-is-not-a-troll-12345678901.mp4"]
findthis= ['1yRMYH8cSFM','asdfghjklgh', '12345678901']


start_idx=namelist[0].find(findthis[0])
# cstring[start_idx:start_idx+11]
#print(start_idx)
print(namelist[0][start_idx:start_idx+11])

#cant use takes too long to process
for h in data: #file
    for a in df_reduced['# YTID']: #ID
        idx=h.find(a) #inside bracket keyword
        name=h[idx:idx+11]
        print(name)

#testing findIDx() function
for h in data:
    print(findIdx())

In [8]:
#match each ID under # YTID column to each file name in downloaded file, returns a name
def findIdx(h): #depends on previously called df_reduced variable
    for a in df_reduced['# YTID']:
        idx=h.find(a) #inside bracket keyword, name returns an index
        if idx!= -1:
            name=h[idx:idx+11]
            return name

In [9]:
#returns a proper file extension format if the file ends with '.part'
def ifPath(h): 
    ext=h.split('.')
    if ext[-1]=='part':
        extention='.'+ ext[-2]
        return extention
    else:
        extention='.'+ ext[-1]
        return extention

#OLD slicing the YT videos I downloaded
os.chdir(r'D:\Alroy youtube proj')
for h in lastslice():
    name=findIdx()
    minidf=df_reduced[df_reduced['# YTID'].isin([name])] #creating single row dataframe for each YT file
    pprint(minidf[['# YTID', ' start_seconds', ' end_seconds']]) #just to print nicer
    filename=minidf.iloc[0]['# YTID'] #taking the first item of the column
    start=minidf.iloc[0][' start_seconds']
    end=minidf.iloc[0][' end_seconds']
    filepath="D:\\Alroy youtube proj\\"+h #calling the file itself
    ext=ifPath()#ext is the extention name
    endpath=f'"D:\\Alroy youtube proj\\Snippets\\{filename}"' #renaming the file
    #ffmpeg_extract_subclip(filepath,start,end, targetname=endpath) #the ultimate slicing function
    video = VideoFileClip(filepath).subclip(start,end)
    print(f'{endpath},\n{ext}\n')
    video.write_videofile(endpath, codec=ext, logger=None)
    video.close() #need close
    print(h, 'has been clipped sucessfully')

In [ ]:
#one time use
#individual slicing of videos, only works for files that's not mkv
os.chdir(r"C:\Users\DSPLAB\Desktop")
name="Cat Speaking-0qZ3tI4nAZE.mkv"
print('C:\\Users\\DSPLAB\Desktop\\'+name)
rename="0qZ3tI4nAZE.mkv"
video = VideoFileClip('C:\\Users\\DSPLAB\Desktop\\'+name).subclip(10,20)
video.write_videofile(rename,logger=None)
video.close() #need close
print(f"{name} has been clipped")

In [10]:
#to convert mkv to mp4, dont use the subprocess function, use 'module'
import subprocess
import ffmpeg

class FFMConverter:
    def convert_mkv_mp4_subprocess(self, input_file, output_file):
        try: #ffmpeg -i test.mkv test.mp4
            command='ffmpeg -i' + input_file+' '+output_file
            subprocess.run(command)
            print("successully converted")
        except:
            print("Video failed to convert mkv to mp4")
            
    def convert_mkv_mp4_module(self, input_file, output_file):
        try:
            stream=ffmpeg.input(input_file)
            stream=ffmpeg.output(stream, output_file)
            ffmpeg.run(stream)
            print("successfully converted")
        except:
            print()
            
ffm=FFMConverter()
#ffm.convert_mkv_mp4_module('C:\\Users\\DSPLAB\\Desktop\\Shuffle vs hp hop--BxQ9nkrId4.mkv', "C:\\Users\\DSPLAB\\Desktop\\newcat"+".mp4") <--- very impt dont delete

In [11]:
#condition to convert MKV/webm file if detected
def convertMKV(h,direct1,direct2): #direct1="C:\\Users\\DSPLAB\\Desktop\\tets2\\", locate the file itself    #direct2=""C:\\Users\\DSPLAB\\Desktop\\", end path   #h is the filename not filepath
    if h.endswith(".webm"):
        bare=h.removesuffix('.webm')  
        clipped=ffm.convert_mkv_mp4_module(direct1+h, direct2+bare+".mp4") #dont fking direct the converted file back to the same folder as the mkv file
        return clipped #returns the file

In [12]:
#creating another condition to remove '-' or '--' in front of filenames
def destroydash(eachfile):
    if eachfile[0]=='-':
        eachfile=eachfile.replace('-','')
    return eachfile

In [38]:
#testing if this problematic video can be clipped. Yes it can. OH BECAUSE THE CLIP is 19 seconds long only BUT THE EXCEL sheet asked to cut 10-20.
video = VideoFileClip("D:\\Alroy youtube proj\\"+"'gr' - Daily Letter (The Electric Company)-EO7sVKW7mY8.mp4").subclip(t_start=10,t_end=16)
video.write_videofile("C:\\Users\\DSPLAB\\Desktop\\walaowhy.mp4", logger=None)
video.close()

In [14]:
#Excel timings exceed video duration
def exceedTime(filename,end):
    directory="D:\\Alroy youtube proj\\"   #"D:\\Alroy youtube proj\\"
    cap=cv2.VideoCapture(directory+filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    time=length/fps
    if end>time: #comparing end_clip time and max duration of video
        end=time
        print("new end time assigned, clip was in fact shorter than clip-endtime ")
    return end

In [15]:
#The Slicing function w/ OSERROR exception, moviepy error exception, create a txt file with its YTID in its place
def slicingCodeOS():
    try:    
        video = VideoFileClip(filepath).subclip(t_start = start,t_end = end)
        video.write_videofile(endpath, logger=None)
        video.close() #need close
    except OSError:
        oof= open(newname+".txt","w+")
        oof.close()

In [25]:
#test OSError    
#creating single row dataframe for each YT file
name=findIdx(r"C:\Users\DPSLAB\Desktop\Lassie - Vivians Rijstadvies - Gezonde lunch inspiratie (2015)---aE2O5G5WE.f248.webm.")
minidf=df_reduced[df_reduced['# YTID'].isin([name])] 
pprint(minidf[['# YTID', ' start_seconds', ' end_seconds']])

#Renaming newfiles for the slicing function
rawname=minidf.iloc[0]['# YTID']
newname=destroydash(rawname)

#Getting start and end timings to slice
start=minidf.iloc[0][' start_seconds']
end=minidf.iloc[0][' end_seconds']

#Getting filepath of each file for the slicing function
filepath=r"C:\Users\DPSLAB\Desktop\Lassie - Vivians Rijstadvies - Gezonde lunch inspiratie (2015)---aE2O5G5WE.f248.webm."

#Getting the extension the video files for the slicing function
ext=ifPath(r"C:\Users\DPSLAB\Desktop\Lassie - Vivians Rijstadvies - Gezonde lunch inspiratie (2015)---aE2O5G5WE.f248.webm.")

#Renaming the output clip
endpath="D:\\Alroy youtube proj\\Snippets\\"+newname+ext

#Continue slicing even if clip-end timing exceed video duration
end=exceedTime("Lassie - Vivians Rijstadvies - Gezonde lunch inspiratie (2015)---aE2O5G5WE.f248.webm.",end)

#ffmpeg_extract_subclip(filepath,start,end, targetname=endpath) <- the OLD ultimate slicing function
#The New Slicing Function
slicingCodeOS()
print(r"C:\Users\DPSLAB\Desktop\Lassie - Vivians Rijstadvies - Gezonde lunch inspiratie (2015)---aE2O5G5WE.f248.webm.", 'has been clipped sucessfully')

        # YTID   start_seconds   end_seconds
2  --aE2O5G5WE             0.0          10.0
new end time assigned, clip was in fact shorter than clip-endtime 
C:\Users\DPSLAB\Desktop\Lassie - Vivians Rijstadvies - Gezonde lunch inspiratie (2015)---aE2O5G5WE.f248.webm. has been clipped sucessfully


In [ ]:
#create a script to check number of snippets and its corresonpding video.no. of videos that has been slice should = number of snippets(find out the reason why)






In [16]:
#The Slicing Code

for h in unslicedset:
    
    #creating single row dataframe for each YT file
    name=findIdx(h)
    minidf=df_reduced[df_reduced['# YTID'].isin([name])] 
    pprint(minidf[['# YTID', ' start_seconds', ' end_seconds']])
    
    #Renaming newfiles for the slicing function
    rawname=minidf.iloc[0]['# YTID']
    newname=destroydash(rawname)
    
    #Getting start and end timings to slice
    start=minidf.iloc[0][' start_seconds']
    end=minidf.iloc[0][' end_seconds']
    
    #Getting filepath of each file for the slicing function
    filepath="D:\\Alroy youtube proj\\"+h
    
    #Getting the extension the video files for the slicing function
    ext=ifPath(h)
    
    #Renaming the output clip
    endpath="D:\\Alroy youtube proj\\Snippets\\"+newname+ext
   
    #Continue slicing even if clip-end timing exceed video duration
    end=exceedTime(h,end)
    
    #ffmpeg_extract_subclip(filepath,start,end, targetname=endpath) <- the OLD ultimate slicing function
    #The New Slicing Function
    
    print(h, 'has been clipped sucessfully')

          # YTID   start_seconds   end_seconds
187  -NmjCyqIavI            30.0          40.0
some freestyle beatboxing  -)--NmjCyqIavI.mp4 has been clipped sucessfully
           # YTID   start_seconds   end_seconds
2650  59k-52vAtVI            16.0          26.0
new end time assigned, clip was in fact shorter than clip-endtime 
nerd rage, keyboard slap-59k-52vAtVI.mp4 has been clipped sucessfully
           # YTID   start_seconds   end_seconds
5211  BkjvDGA8_94            60.0          70.0
The Homer Howl-BkjvDGA8_94.mp4 has been clipped sucessfully
          # YTID   start_seconds   end_seconds
546  0AhX-ReXovI           120.0         130.0
Zaroori Tha by Shiv-0AhX-ReXovI.mp4 has been clipped sucessfully
           # YTID   start_seconds   end_seconds
6066  E14EBYmiGWg            30.0          40.0
Taylor Swift Red Music Video Official Safe And Sound Eyes Open Begin Again Movie Soundtrack AMA-E14EBYmiGWg.mp4 has been clipped sucessfully
           # YTID   start_seconds   end_second

ZeroDivisionError: float division by zero

In [18]:
#testing the single file slicing function works maah!!!!! so why cant i batch process!!!! 
# SOLVED: CANNOT FKING HAVE '-' in the new filename!!!!!!! Now focusing on a new name or to bypass this issue
#testing the file that has error
newname="-A0MTi2NMGI"
ext=".mp4"
endpath="C:\\Users\\DSPLAB\\Desktop\\"+newname+ext
video = VideoFileClip("C:\\Users\\DSPLAB\\Desktop\\#02 Stock Car _ 2016 - Teaser Velopark (F3 _ CBT _ Marcas)--A0MTi2NMGI.mp4").subclip(5,10)
video.write_videofile(endpath, logger=None)
video.close() #need close
print('has been clipped sucessfully')

OSError: [Errno 32] Broken pipe

MoviePy error: FFMPEG encountered the following error while writing file -A0MTi2NMGITEMP_MPY_wvf_snd.mp3:

b"Unrecognized option 'A0MTi2NMGITEMP_MPY_wvf_snd.mp3'.\r\nError splitting the argument list: Option not found\r\n"

In case it helps, make sure you are using a recent version of FFMPEG (the versions in the Ubuntu/Debian repos are deprecated).

In [31]:
#one time use
#Checking if file extension= .mkv(which fucks the slicing function up), converting .mkv to .mp4
#CONVERTING ALL CURRENT MKV/.webm FILES TO MP4
for i in os.listdir(r'D:\Alroy youtube proj'):
    convertMKV(i, 'D:\\Alroy youtube proj\\', 'D:\\Alroy youtube proj\\convertedMp4\\') #Need create convertMp4 folder

successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfully converted
successfull

def matchingFn(): #returns a True/ False value
    in_data=0
    #in_YTID=0
    YTID=df_reduced['# YTID'].tolist()
    for u in YTID:
        counter=0
        match=bool(re.findall(YTID[length], data[in_data], re.IGNORECASE)) #if a match is found print true
        if match==False:
            counter+=1
            continue
        else: 
            return data[in_data], counter
            print(data[in_data])
            print(counter)
            restart for loop

print(matchingFn())

for i in range(len(data)): #goes through all files in data list and goes through all ID's in # YTID column and Produces True/False
    YTID=df_reduced['# YTID'].tolist()
    print(YTID[i])
    print(data[i])
    match=bool(re.findall(YTID[i], data[i], re.IGNORECASE)) #if a match is found print true
    print(match)

#for row in range(len(df_reduced)): #to pick each YTID, start, endtime respectively in the df_reduced correctly
#     if filename is in (parse through each item in 'data'_list:
#         extract datafile (can define a function for this)
result=matchingFn() #result[0] = the youtube file itself #result[1] = the index of the # YTID in the reduced dataframe
filename=df_reduced.iloc[result[1]]['# YTID'] #if youtubefile matches with dataframe YTID, do:......, but how to find the row number
start=df_reduced.iloc[result[1]][' start_seconds']
end=df_reduced.iloc[result[1]][' end_seconds']
print(filename, start, end) #it works, one at a time
ytslice(result[0], start, end, filename+'.mp4')

#testing cell for match functions
YTID=df_reduced['# YTID'].tolist()
print(YTID[3])
print(data[6])
match=bool(re.findall(YTID[3], data[6], re.IGNORECASE)) #if a match is found print true
print(match)

imtag = re.match(r'<img.*?>', line).group(0)

ytslice(match,5,10, 'testing.mp4')

# Use of re.search() Method
for key in range(len(data)):
    item=re.search(YTID[3], data[key], re.IGNORECASE)
    print(item) #it returns an object!!!!
    if item==None:
        continue
    a=item.expand(\)
    print(a)
#     ytslice(item,5,10,'testing.mp4')

test1=data[0].str.contains('--PJHxphWEs')
test1

newlist=YTID.str.contains('|'.join(data)) ###HOW TO LINK KEYWORDS AND ASSIGN THEM AS AN ITERATATBLE VARIABLE TO PUT INTO THE SLICE FUNCTION
# FIRST I MATCH THEN I EXTRACT THE FILE AS VARIABLE

if YTID[0]== 'keyword' in data(s):
    ytslice(data[_],start[0],end[0],YTID[0]+'.mp4')

def ytslice(files,start,end,targetname): #files= the YT mp4 itself, start is the seconds, end is the seconds, filename is the ___.mp4 name u want
    y=ffmpeg_extract_subclip(files, start,end, targetname=targetname)
    return y

for u in range(len(files)): #toslice the YT videos
    ytslice(files[u],starttime[u],endtime[u],fibrelist[u]+'.mp4') #ytslice returns Nonetype
    print(files[u], 'is finished converting')
    shutil.move(fibrelist[u]+'.mp4',r'D:\Alroy youtube proj\Snippets') #move to a new directory
    #DONE

#to reduce the dataframe to our YTIDS that has successfully downloaded
newfilename=[]
startTime=[]
endTime=[]
for j in range(len(orange)):
    sieve =  startend['# YTID']==orange[j]
    blahblah=startend[sieve] #blahblah is a single row with column headings dataframe
    #pprint(blahblah)
#     newfilename=blahblah['# YTID']
#     startTime=blahblah[' start_seconds']
#     endTime=blahblah[' end_seconds']
#     pprint(newfilename)
#     pprint(startTime)
#     pprint(endTime)
#creating different lists: newfilename, startTime, endTime consisting of all the values i need in index order
    newfilename.append(blahblah['# YTID'])
    startTime.append(blahblah[' start_seconds'])
    endTime.append(blahblah[' end_seconds'])

startend.iloc[:5] #getting rows from dataframe

for index, row in datafram.iterrows():
    print(row['begin'], row['end'])

starttime=list(startend[' start_seconds']) #remember to put the space in front of start_seconds
print(starttime)

os.getcwd()

os.chdir(r'D:\Alroy youtube proj')

endtime=list(startend[' end_seconds'])
print(endtime)

#useless because file list unordered
#to just get the "file" term in the slice function
files=os.listdir('D:\Alroy youtube proj')
pprint(files) #if this cell is rerunned, the file list will contain all files again

files.remove('YTcommand_prompt.txt') #dont run this cell again, to delete the text file off the list

files.remove('Snippets') #dont run this cell again to delete snippets folder off the list

pprint(files) #the only YT files exixsitng/downloaded in my folder (does not contain IDs that failed to download)

#cropping the 'files' names, since they all have 11 characters each
fibrelist=[]
for z in files:
    paper=z.split('.')
    print(paper) #consist of two parts
    wood=paper[0] #taking the qst object of paper which is the name, not the .mp4
    fibre=wood[-11:]#geting the last 11 characters of each name
    print(fibre) #rmbr for loop is one at a time
    fibrelist.append(fibre) #adding to an empty list
print(fibrelist) #we have a list of youtube names that were sucessfully downlaoded

pprint(files)

#try renaming the clip to YTID name
for files in os.listdir('D:\Alroy youtube proj'): #i got each individual element in the dataframe
    if files.endswith('.txt'): #skip my text document containing cmd texts
        continue #works until heres
    for index, row in datafram.iterrows(): #to call the start and end timings
        ffmpeg_extract_subclip(files, row['begin'],row['end'], targetname=files+'.mp4') #slice video clip, it rewrite the same file if oyu dont rename it properly
        break
    print(files, 'video is done processing!')

os.listdir('D:\Alroy youtube proj')

index=0
for start in datafram['begin']: #iterating through start_seconds column
    return start
for end in datafram['end']: #iterating through end_seconds column
    return end
os.listdir('D:\Alroy youtube proj') #calling all videos
for filename in os.listdir('D:\Alroy youtube proj'):
    if filename.endswith('.txt'): #skip my text document containing cmd texts
        continue
    return filename
ffmpeg_extract_subclip(filename[], start, end, targetname=filename+".mp4") #slice video clip

import pandas as pd

df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})
print(df)
print('\n')
for index, row in df.iterrows():
    print(row['c1'], row['c2'])

-o ~/C:/Users/DSPLAB/Desktop/Alroy Data/Youtube-dl proj/yt vids raw/.mp4

df=pd.read_csv('balanced_train_segments_propertext.xlsx', delim_whitespace=True)
df

onecol=df['YTID']
onecol

result = onecol.str.strip(', ')
#orders['product_type'].str.strip('product_type ')
result